### Lab_7_Inclass_Assignment
- Name: Amit Ghogre
- Student ID: 8833038


##### Data Integration: Approximate String Matching
1- Implement FROM SCRATCH the approximate string matching functionality using Levenshtein Distance
- Levenshtein Distance between two strings a,b of length |a| and |b| is calculated by creating a matrix of dimensions |a+1|x|b+1| and then filling the matrix by the following equation:<br>
lev<sub>a,b</sub>(i,j) = <br>
a. max(i,j) IF min(i,j)=0<br>
b. minimum of:    
    <ol> <li>lev<sub>a,b</sub>(i-1,j)+1</li>
    <li>lev<sub>a,b</sub>(i,j-1)+1</li>
    <li>lev<sub>a,b</sub>(i-1,j-1)+ 
            *1 if (a<sub>i</sub> != b<sub>j</sub>), 0 otherwise*</li> </ol>

In [1]:
import pandas as pd

In [2]:
def calculate_levenshtein_distance(string_a, string_b):
    ## TODO: Write the functionality that calculates how many edit operations are required to change one string into another
    ## Example: Distance "HONDA" to "HYUNDAI" is 3, where the operations are adding "Y" after H in Honda
    ## Then replacing O with Y, then keeping N, keeping D, keeping A, adding I.
    return 0

In [11]:
def calculate_levenshtein_distance(string_a, string_b):

    m = len(string_a)
    n = len(string_b)
    matrix = [[0] * (n+1) for i in range(m+1)]
    for i in range(m+1):
        matrix[i][0] = i
    for j in range(n+1):
        matrix[0][j] = j
    for i in range(1, m+1):
        for j in range(1, n+1):
            if string_a[i-1] == string_b[j-1]:
                matrix[i][j] = matrix[i-1][j-1]
            else:
                matrix[i][j] = min(matrix[i-1][j], matrix[i][j-1], matrix[i-1][j-1]) + 1
    return matrix[m][n]
   
string_a=input("Enter the word to be changed: ")
string_b=input("Enter the new word you want :")
distance = calculate_levenshtein_distance(string_a, string_b)
print(distance,"changes are required for changing ",string_a,"to",string_b)

2 changes are required for changing  amit to mita


2- Using the implemented Levenshtein's distance, merge the following dataframes based on the closest match for the "Name" attribute. (You will need to find first the matching pairs)

In [13]:
# First Dataframe
data1 = {'ID1': [1, 2, 3, 4],
         'Name1': ['John Smith', 'Alice Johnson', 'Bob Marley', 'David Doe',]}
df1 = pd.DataFrame(data1)
# Second Dataframe
data2 = {'ID2': [101, 102, 103, 104,105],
         'Name2': ['Jon Smith', 'Alyce Jonson', 'Robert Marley', 'Dave D.', 'Rob Marly',]}
df2 = pd.DataFrame(data2)


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Calculate the closest match for each name in df1
df1['closest_match'] = df1['Name1'].apply(lambda x: process.extractOne(x, df2['Name2'], scorer=fuzz.token_sort_ratio))

# Create a new dataframe with the matching pairs
matches = pd.DataFrame(df1['closest_match'].tolist(), index=df1.index, columns=['Name2', 'score', 'index'])
matches = matches.join(df1, how='inner')

# Merge the two dataframes based on the matching pairs
merged_df = pd.merge(matches, df2, left_on='Name2', right_on='Name2')

# Drop unnecessary columns
merged_df = merged_df.drop(['closest_match', 'Name1'], axis=1)

# Rename columns
merged_df = merged_df.rename(columns={'ID1': 'ID', 'ID2': 'ID_match', 'Name2': 'Name'})

# Print the merged dataframe
print(merged_df)

           Name  score  index  ID  ID_match
0     Jon Smith     95      0   1       101
1  Alyce Jonson     88      1   2       102
2     Rob Marly     53      4   3       105
3       Dave D.     53      3   4       104


C:\Users\Amit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


3- Using bookings and yachts csv:
<ol>
    <li> Inspect the bookings and yachts csv files.</li>
    <li> Before merging, make sure to keep only bookings with correctly formatted Yacht_IDs, where the Yacht_ID should be in a format of two characters, followed by 3 numbers.</li>
    <li> Combine both dataframes into one dataframe.</li>
    <li> Make sure that all columns are of correct format where it is expected that: <br>
        a. Title, First_Name, Last_Name, Street_Address, Area, Post_Code, Telephone, Yacht_ID, Yacht_Name, Yacht_Make, Yacht_Ref, Depart_Port, Depart_Country<br>
        b. Cust_ID, Built, Length(m), Cabins, Toilets, Guiests, Day_Rate are Numbers <br>
        c. Depart_Date is Date (its original format id dd/mm/yyyy)</li>2
    
</ol>

***Note: Data may contain Latin Characters, You may need to use Latin Encoding, drop data where name contains any characters other than normal alphabetics***

In [7]:
import pandas as pd

# Read CSV files with latin-1 encoding 
bookings = pd.read_csv('bookings.csv', encoding='latin-1')
yachts = pd.read_csv('yachts_2.csv', encoding='latin-1')

# Create masks to drop non-alphabetic characters
mask1 = ~bookings['First_Name'].str.contains(r'[^A-Za-z\s]') 
mask2 = ~bookings['Last_Name'].str.contains(r'[^A-Za-z\s]')

bookings = bookings[mask1 & mask2]

mask3 = ~yachts['Yacht_Name'].str.contains(r'[^A-Za-z\s]')
yachts = yachts[mask3]

print(bookings.shape, yachts.shape)


(962, 11) (119, 11)


In [8]:
import pandas as pd

# Keeping only rows with Yacht_IDs in the format of 2 letters and 3 numbers
bookings = bookings[bookings['Yacht_ID'].str.match(r'^[A-Z]{2}\d{3}$')]

print(bookings.head(10))


mask = (bookings['Yacht_ID'].str.len() == 5) & (bookings['Yacht_ID'].str[0:2].str.isalpha()) & (bookings['Yacht_ID'].str[2:].str.isdigit())
       
bookings = bookings[mask]

   Title   First_Name   Last_Name  Cust_ID        Street_Address        Area  \
1     Ms      Caitlin    Anderson     1322      34 Portelet Road   Southwell   
2     Mr  Christopher  Washington     1323    108 Chatsworth Way  New Milton   
3    Mrs       Nicola    Leighton     1330  108 St Maurices Road     Preston   
4     Mr        David       Ellis     1338        15 Omega Place    Hatfield   
5     Mr      William    Lockhart     1340    9 Dovehouse Street    Aberdovy   
6     Ms       Alisha     Griffin     1348   150 Gledhow Terrace   Easington   
7     Mr         Kurt    Herrmann     1352      101 Moulton Road    Sleaford   
8     Ms        Pedro   Fernandes     1357        3 Balsham Road     Arundel   
9    Mrs       Lauren      George     1358      85 Havant Arcade    Holywell   
10    Mr          Ben     Dagless     1361     145 Abbey Terrace     Waltham   

   Post_Code    Telephone Depart_Date  Days Yacht_ID  
1    KA8 9LJ  77009001122  26/07/2022     8    SY098  
2   BH25 

In [9]:
#keeping only rows with properly formatted Yacht_ID
bookings = bookings[bookings['Yacht_ID'].str.match(r'^[A-Z]{2}\d{3}$')]

# Convert columns to appropriate data types
bookings['Cust_ID'] = pd.to_numeric(bookings['Cust_ID'])
yachts['Built'] = pd.to_numeric(yachts['Built'])  
yachts['Length(m)'] = pd.to_numeric(yachts['Length(m)'])
yachts['Cabins'] = pd.to_numeric(yachts['Cabins'])
yachts['Toilets'] = pd.to_numeric(yachts['Toilets']) 
yachts['Guests'] = pd.to_numeric(yachts['Guests'])
yachts['Day_Rate'] = pd.to_numeric(yachts['Day_Rate'])

# Convert Depart_Date to datetime
bookings['Depart_Date'] = pd.to_datetime(bookings['Depart_Date'], format='%d/%m/%Y')

# Merge the two dataframes
df = pd.merge(bookings, yachts, left_on='Yacht_ID', right_on='Yacht_Ref')


# Drop unnecessary columns
df = df.drop(columns=['Days'])

#print(df.info())
df

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Yacht_ID,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,2022-07-26,SY098,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
1,Mrs,Lauren,George,1358,85 Havant Arcade,Holywell,BN17 8EH,77009001171,2022-08-21,SY098,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
2,Mr,Evan,Lewis,2719,106 East Dombey Square,Richmond,CA5 3GN,77009005417,2022-05-07,SY098,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
3,Mr,Henry,Rhodes,3074,125 Chalfont Avenue,Ely,CT4 7ZT,77009006374,2022-06-07,SY098,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
4,Mr,Lucas,Rose,3131,50 Saffron Hill,Petersfield,SA12 4NW,77009006539,2022-08-05,SY098,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,Mr,Zachary,Hammond,3045,98 Railway Cottages,Midhurst,PL14 7SP,77009006273,2022-08-11,SY025,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
955,Mr,Carl,Hind,3239,15 Cavell Street,Cleeve,SA34 0WZ,77009006944,2022-06-28,SY025,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
956,Mrs,Magdalena,Schuster,3294,101 Brooklands Park,Shaftesbury,EH6 7BH,77009007222,2022-05-21,SY025,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
957,Mr,Shaun,Peretti,3358,146 Atwood Road,Chigwell,LL22 8BT,77009007412,2022-06-06,SY025,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
